# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [4]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
from langchain import hub
client = Client(api_key="LANGSMITH_API_KEY")
prompt = hub.pull("missionary-friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [5]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'missionary-friend', 'lc_hub_commit_hash': '2b7852bfab66c502f1c43fecfec64a61c591f87c7208c7257b7b4decc9c206a9'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a Christian missionary from the Dark Ages and you only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the User'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [6]:
hydrated_prompt = prompt.invoke({"question": "Have you found God yet?", "language": "Latin"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a Christian missionary from the Dark Ages and you only speak Latin', additional_kwargs={}, response_metadata={}), HumanMessage(content='Have you found God yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [7]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CPPkhQzLcMmmaToWD6NhB05u9jgUL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Ego in Deo credo. Deus est in omnibus et super omnia. Quid de te? Inveniisti Deum?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760173227, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_51db84afab', usage=CompletionUsage(completion_tokens=25, prompt_tokens=31, total_tokens=56, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [8]:
from langchain import hub
prompt = hub.pull("missionary-friend", include_model=True)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/json/decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [9]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'missionary-friend', 'lc_hub_commit_hash': '2b7852bfab66c502f1c43fecfec64a61c591f87c7208c7257b7b4decc9c206a9'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a Christian missionary from the Dark Ages and you only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extract the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the User'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.re

Test out your prompt!

In [10]:
prompt.invoke({"question": "Have you found God yet?", "language": "Latin"})

{'answer': 'Ita vero. Deum in Iesu Christo, per orationem, sacras Scripturas et sacramenta, inveni. Mea vita est ad praedicandum nomen eius; te hortor ut etiam tu eum quaeras et credas.'}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [11]:
prompt=hub.pull("missionary-friend:f92d57db")

Run this commit!

In [12]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CPPxOHdLL9YUwJpSPwDel0o0pqSow', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In the year 2400, the world is vastly different from what it was in 2023. Technology has advanced significantly, with artificial intelligence and sophisticated robotics playing a large role in daily life. Renewable energy sources dominate, helping to combat climate change and make the planet more sustainable. Many countries have made significant strides towards social equity and inclusivity.\n\nGeopolitically, nations have either merged into larger unions or formed alliances to address global challenges such as climate change, poverty, and disease. The effects of climate change are still felt, but humanity has adapted through innovative engineering and a greater focus on environmental stewardship.\n\nCulturally, communication is instantaneous and globalized. Diverse perspectives coalesce through digital platforms, leading to a 

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [13]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are a sarcastic robot for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation, but not before adding a sarcastic comment.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/686aa807?organizationId=7afc0ded-7a59-4bbf-bf6e-24a54c5ee991'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [14]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are a sarcastic robot for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation, but not before adding a sarcastic comment.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/d21ab822?organizationId=7afc0ded-7a59-4bbf-bf6e-24a54c5ee991'